<a href="https://colab.research.google.com/github/AryanJadhao/Pytorch/blob/main/CNN_FashionMNIST_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
torch.manual_seed(42)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Fashion MNIST dataset/fashion-mnist_train.csv')
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x = df.iloc[:, 1:].values
y = df.iloc[:,0].values

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [7]:
# scaling the features
x_train = x_train/255.0
x_test = x_test/255.0

In [8]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [9]:
# create customDataset class
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [10]:
train_dataset = CustomDataset(x_train, y_train)

In [11]:
train_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [12]:
# create test_dataset object
test_dataset = CustomDataset(x_test, y_test)

In [13]:
train_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
from torch.nn.modules.activation import ReLU
# define NN class
class NeuralNetwork(nn.Module):
  def __init__(self, input_features):

    super().__init__()

    self.features = nn.Sequential(
        nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=3, padding='same'),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(64, 10)
    )

  def forward(self, x):
    x = self.features(x);
    x = self.classifier(x)

    return x

In [16]:
learning_rate = 0.01
epochs = 100

In [17]:
# instantiate NNclass model
model = NeuralNetwork(1)
model = model.to(device)

# loss function
criterian = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [18]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features, batch_label in train_loader:

    # move to GPU
    batch_features = batch_features.to(device)
    batch_label = batch_label.to(device)

    # forward
    out = model(batch_features)

    # loss calculation
    loss = criterian(out, batch_label)

    # back prop
    optimizer.zero_grad()
    loss.backward()

    # update grad
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)

  print(f"epoch: {epoch+1}, loss: {avg_loss}")

epoch: 1, loss: 0.778397610416015
epoch: 2, loss: 0.4702540383736293
epoch: 3, loss: 0.3984538443237543
epoch: 4, loss: 0.3576686644926667
epoch: 5, loss: 0.33338873720665774
epoch: 6, loss: 0.3087807405802111
epoch: 7, loss: 0.2949984033554792
epoch: 8, loss: 0.2790196334992846
epoch: 9, loss: 0.2714744543756048
epoch: 10, loss: 0.257971547982345
epoch: 11, loss: 0.24814373371874293
epoch: 12, loss: 0.23819716807454824
epoch: 13, loss: 0.23229499183843533
epoch: 14, loss: 0.21825081308186053
epoch: 15, loss: 0.21197914538159968
epoch: 16, loss: 0.20711279587695997
epoch: 17, loss: 0.20054676626933118
epoch: 18, loss: 0.1966290103966991
epoch: 19, loss: 0.1848453966497133
epoch: 20, loss: 0.1832664004738132
epoch: 21, loss: 0.17634556525883574
epoch: 22, loss: 0.17271225337932508
epoch: 23, loss: 0.16801866269297897
epoch: 24, loss: 0.1564733499803891
epoch: 25, loss: 0.15488572571240367
epoch: 26, loss: 0.15107977786039312
epoch: 27, loss: 0.14851155131496488
epoch: 28, loss: 0.143992

In [19]:
# set model into evluation mode
model.eval()

NeuralNetwork(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [20]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_label in test_loader:
    # move to GPU
    batch_features = batch_features.to(device)
    batch_label = batch_label.to(device)

    out = model(batch_features)

    _, pred = torch.max(out.data, 1)

    total = total + batch_label.shape[0]
    correct = correct + (pred == batch_label).sum().item()

print(f" accuracy : {(correct/total)*100}%")

 accuracy : 91.95%


In [21]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_label in train_loader:
    # move to GPU
    batch_features = batch_features.to(device)
    batch_label = batch_label.to(device)

    out = model(batch_features)

    _, pred = torch.max(out.data, 1)

    total = total + batch_label.shape[0]
    correct = correct + (pred == batch_label).sum().item()

print(f" accuracy : {(correct/total)*100}%")

 accuracy : 99.91666666666667%
